# Amazon Bedrock and KDB.AI

## Document Summary using Amazon Bedrock and KDB.AI

The following examples demonstrate some of the ways you can interact with KDB.AI and Amazon Bedrock through LangChain.

**Authors: Chintan Sanghavi**

# Amazon Bedrock and KDB.AI

> [Amazon Bedrock](https://aws.amazon.com/bedrock/) is the easiest way to build and scale generative AI applications with foundation models from third party model providers and your own custom models

> [KDB.AI](https://kdb.ai/) is a powerful knowledge-based vector database and search engine that allows you to build scalable, reliable AI applications, using real-time data, by providing advanced search, recommendation and personalization.

To access your end point and API keys, [sign up to KDB.AI here](https://kdb.ai/get-started/).

To set up your development environment, follow the instructions on the [KDB.AI pre-requisites page](https://code.kx.com/kdbai/pre-requisites.html).


## Install required Packages

In [2]:
!pip install -q --upgrade pip
!pip install -q langchain-aws 
!pip install -q langchain-community boto3 
!pip install -q pypdf 

## Import required packages

In [3]:
import os
import time

import kdbai_client as kdbai
import pandas as pd
import requests
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import KDBAI
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# AWS related Langchain Packages
from langchain_aws import ChatBedrockConverse
from langchain_aws import BedrockEmbeddings
from langchain_aws import ChatBedrock
#from langchain_community.embeddings import BedrockEmbeddings
#from langchain_aws import BedrockLLM

<div class="alert alert-block alert-warning">
  <b>Warning:</b> Use blue boxes (alert-info) for tips and notes.
</div>

In [4]:
KDBAI_ENDPOINT = "https://cloud.kdb.ai/instance/2dwnisr9hd"
KDBAI_API_KEY = "f4a106ddc6-HXoJJqjslErpNo+x9nEtlxfmU+vyXs9Eol0X8JJF49KnfBF1UWQw83S06yva/qZUN6qDIa1Uw2ZBOv9m"

In [5]:
TEMP = 0.0
K = 3

## Create a KBD.AI Session

In [6]:
print("Create a KDB.AI session...")
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)

Create a KDB.AI session...


## Create a table

In [7]:
print('Create table "documents"...')
schema = {
    "columns": [
        {"name": "id", "pytype": "str"},
        {"name": "text", "pytype": "bytes"},
        {
            "name": "embeddings",
            "pytype": "float32",
            "vectorIndex": {"dims": 1536, "metric": "L2", "type": "hnsw"},
        },
        {"name": "tag", "pytype": "str"},
        {"name": "title", "pytype": "bytes"},
    ]
}
table = session.create_table("documents", schema)

Create table "documents"...


In [8]:
%%time
URL = "https://www.conseil-constitutionnel.fr/node/3850/pdf"
PDF = "Déclaration_des_droits_de_l_homme_et_du_citoyen.pdf"
open(PDF, "wb").write(requests.get(URL).content)

CPU times: user 19.3 ms, sys: 6.52 ms, total: 25.8 ms
Wall time: 1.27 s


562978

## Read a PDF

In [9]:
%%time
print("Read a PDF...")
loader = PyPDFLoader(PDF)
pages = loader.load_and_split()
len(pages)

Read a PDF...
CPU times: user 352 ms, sys: 6.94 ms, total: 359 ms
Wall time: 462 ms


3

## Create a Vector Database from PDF Text

In [10]:
%%time
print("Create a Vector Database from PDF text...")
embeddings = BedrockEmbeddings()
texts = [p.page_content for p in pages]
metadata = pd.DataFrame(index=list(range(len(texts))))
metadata["tag"] = "law"
metadata["title"] = "Déclaration des Droits de l'Homme et du Citoyen de 1789".encode(
    "utf-8"
)
vectordb = KDBAI(table, embeddings)
vectordb.add_texts(texts=texts, metadatas=metadata)

Create a Vector Database from PDF text...
CPU times: user 86.1 ms, sys: 16.7 ms, total: 103 ms
Wall time: 843 ms


['da226f59-4e41-4b32-8bbb-3f2e31322a2a',
 '7e9dfe93-e029-43c7-9779-1b86fd28a18f',
 'f5b25424-916a-412d-80ff-91a26a804455']

## Create LangChain Pipeline

In [11]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=TEMP),
)

In [12]:
from langchain_aws import ChatBedrockConverse
llm_converse = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=TEMP,
    max_tokens=None,
)

In [13]:
%%time
print("Create LangChain Pipeline...")
qabot = RetrievalQA.from_chain_type(
    chain_type="stuff",
    llm=llm_converse, # Change this based on Converse or non-converse LLM
    retriever=vectordb.as_retriever(search_kwargs=dict(k=K)),
    return_source_documents=True,
)

Create LangChain Pipeline...
CPU times: user 90.5 ms, sys: 28.4 ms, total: 119 ms
Wall time: 128 ms


## Summarize the document in English

In [14]:
%%time
Q = "Summarize the document in English:"
print(f"\n\n{Q}\n")
print(qabot.invoke(dict(query=Q))["result"])



Summarize the document in English:

This is the Declaration of the Rights of Man and of the Citizen from 1789, one of the fundamental documents of the French Revolution. It proclaimed the basic rights and freedoms of citizens, including:

- Equality before the law and equal rights for all men (Article 1)
- The rights of liberty, property, security, and resistance to oppression (Article 2) 
- Sovereignty residing in the nation, not any individual or group (Article 3)
- Freedom to do anything that does not harm others (Article 4)
- Laws can only prohibit actions harmful to society (Article 5)
- Equal application of the law, equal eligibility for public offices (Article 6)
- Due process, no arbitrary arrest or detention (Articles 7-9)
- Freedom of religion and opinion (Articles 10-11)
- Taxation only with consent and oversight (Articles 13-14)
- Accountability of public officials (Article 15)
- Property rights, with ability to take for public necessity with compensation (Article 17)

It

## Query the Data

In [15]:
%%time
Q = "Is it a fair law and why ?"
print(f"\n\n{Q}\n")
print(qabot.invoke(dict(query=Q))["result"])



Is it a fair law and why ?

The Declaration of the Rights of Man and of the Citizen of 1789 laid down important principles of liberty, equality, and individual rights that were revolutionary and progressive for its time. However, by today's standards, there are some aspects that could be seen as unfair or incomplete:

- It does not explicitly grant rights to women, referring only to the "rights of man" (although some have argued this was intended in a gender-neutral way).

- It does not abolish slavery or address the rights of minorities and marginalized groups.

- Some of the language around property rights could be seen as prioritizing the rights of property owners over others.

- There are limitations on freedom of religion and expression that may be viewed as too restrictive today.

That said, for its historical context, it represented a major step forward in codifying basic human rights and limiting the power of the state over individuals. Many of its core principles around libe

In [16]:
%%time
Q = "What are the rights and duties of the man, the citizen and the society ?"
print(f"\n\n{Q}\n")
print(qabot.invoke(dict(query=Q))["result"])



What are the rights and duties of the man, the citizen and the society ?

According to the French Declaration of the Rights of Man and of the Citizen of 1789, some of the key rights and duties outlined are:

Rights of Man:
- Men are born and remain free and equal in rights (Article 1)
- Natural and imprescriptible rights include liberty, property, security, and resistance to oppression (Article 2)
- Liberty consists of being able to do anything that does not harm others (Article 4)
- The law can only prohibit actions harmful to society (Article 5)
- No one shall be accused, arrested or detained except in cases determined by law (Article 7)
- The free communication of thoughts and opinions is one of the most precious rights (Article 11)

Rights of the Citizen:
- All citizens have the right to participate personally or through representatives in the formation of law (Article 6)
- All citizens are equal before the law and eligible for public positions based on merit (Article 6)

Duties 

In [17]:
%%time
Q = "Is this law practical ?"
print(f"\n\n{Q}\n")
print(qabot.invoke(dict(query=Q))["result"])



Is this law practical ?

The Declaration of the Rights of Man and of the Citizen of 1789 was a foundational document that established important principles of human rights and civil liberties. However, putting all of its ideals into full practice proved challenging at the time and in the years that followed.

Some key points about the practicality and implementation of this declaration:

- It set forth admirable goals like equality before the law, freedom of speech and religion, due process, and the consent of the governed for taxation. However, fully realizing these rights took time and struggle.

- The declaration was more of a statement of philosophy rather than a detailed legal code that could be easily enforced across France.

- Certain rights like property rights (Article 17) came into conflict with the more radical factions during the French Revolution.

- While influential, the declaration initially lacked solid constitutional mechanisms to ensure its principles were upheld by

## Clean up the Documents table

In [18]:
# Clean up KDB.AI "documents" table and index for similarity search
# so this notebook could be played again and again
session.table("documents").drop()

True